In [4]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rissu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

In [26]:
file=open('intents.json')

In [27]:
intents=json.load(file)

In [28]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of Lon

In [158]:
words=[]
classes=[]
documents=[]
ignore=['?']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w=nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [82]:
wordd=[]
for w in words:
    if w not in ignore:
        wordd.append(stemmer.stem(w.lower()))
        

In [109]:
wordd=sorted(list(set(wordd)))

In [87]:
len(documents),len(classes),len(wordd)

(31, 9, 57)

In [212]:
training=[]
output=[]
output_empty=[0]*len(classes)
for doc in documents:
    bag=[]
    pattern_words=doc[0]
    pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
    for w in wordd:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    training.append([bag,output_row])
random.shuffle(training)
training=np.array(training)
train_x=list(training[:,0])
train_y=list(training[:,1])

In [152]:
# resetting underlying graph data
tf.reset_default_graph()

# building neural network

net=tflearn.input_data(shape=[None,len(train_x[0])])
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,len(train_y[0]),activation='softmax')
net=tflearn.regression(net)

# defining model and setting up tensorboard
model=tflearn.DNN(net,tensorboard_dir='tflearn_logs')

# start training
model.fit(train_x,train_y,n_epoch=1000,batch_size=8,show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.00127 | time: 0.005s
| Adam | epoch: 1000 | loss: 0.00127 - acc: 1.0000 -- iter: 24/31
Training Step: 4000  | total loss: 0.00132 | time: 0.006s
| Adam | epoch: 1000 | loss: 0.00132 - acc: 1.0000 -- iter: 31/31
--
INFO:tensorflow:C:\Users\rissu\A_DataScience\ML_Model\Chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [153]:
len(train_x[0])

57

In [154]:
len(train_y[0])

9

In [157]:
import pickle
pickle.dump({'words':words,
             'classes':classes,
             'train_x':train_x,
             'train_y':train_y},open('training_data','wb'))